In [1]:
# 查看GPU配置
# Check GPU configuration
!nvidia-smi

Sat May  4 10:24:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1660 ...    Off | 00000000:01:00.0  On |                  N/A |
| 25%   41C    P8              15W / 125W |    442MiB /  6144MiB |      5%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pwd

/home/lavi/Documents/myprj/VITS-fast-fine-tuning


In [3]:
#@title STEP 1 复制代码库并安装运行环境
#@markdown #STEP 1 (6 min)
#@markdown ##复制代码库并安装运行环境
#@markdown ##Clone repository & Build environment

# !git clone https://github.com/Plachtaa/VITS-fast-fine-tuning.git
!pip install --force-reinstall soundfile
!pip install --force-reinstall gradio==3.48.0
!pip install imageio==2.4.1
!pip install --upgrade youtube-dl
!pip install moviepy
%cd VITS-fast-fine-tuning

!pip install --no-build-isolation -r requirements.txt
!pip install --upgrade numpy
!pip install --upgrade --force-reinstall numba
!pip install --upgrade Cython

!pip install --upgrade pyzmq
!pip install pydantic==1.10.4
!pip install ruamel.yaml
!pip install git+https://github.com/openai/whisper.git

# build monotonic align
%cd monotonic_align/
!mkdir monotonic_align
!python setup.py build_ext --inplace
%cd ..
!mkdir pretrained_models
# download data for fine-tuning
# !wget https://huggingface.co/datasets/Plachta/sampled_audio4ft/resolve/main/sampled_audio4ft_v2.zip
# !unzip sampled_audio4ft_v2.zip
# create necessary directories
!mkdir video_data
!mkdir raw_audio
!mkdir denoised_audio
!mkdir custom_character_voice
!mkdir segmented_character_voice

  Using cached soundfile-0.12.1-py2.py3-none-manylinux_2_31_x86_64.whl.metadata (14 kB)
  Using cached cffi-1.16.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached soundfile-0.12.1-py2.py3-none-manylinux_2_31_x86_64.whl (1.2 MB)
Using cached cffi-1.16.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (443 kB)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)
  Attempting uninstall: pycparser
    Found existing installation: pycparser 2.22
    Uninstalling pycparser-2.22:
      Successfully uninstalled pycparser-2.22
  Attempting uninstall: cffi
    Found existing installation: cffi 1.16.0
    Uninstalling cffi-1.16.0:
      Successfully uninstalled cffi-1.16.0
  Attempting uninstall: soundfile
    Found existing installation: soundfile 0.12.1
    Uninstalling soundfile-0.12.1:
      Successfully uninstalled soundfile-0.12.1
  Using cached gradio-3.48.0-py3-none-an

/home/lavi/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


  Using cached Cython-0.29.21-py2.py3-none-any.whl.metadata (2.6 kB)
Using cached Cython-0.29.21-py2.py3-none-any.whl (974 kB)
  Attempting uninstall: Cython
    Found existing installation: Cython 3.0.10
    Uninstalling Cython-3.0.10:
      Successfully uninstalled Cython-3.0.10
  Using cached numba-0.59.1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.7 kB)
  Using cached llvmlite-0.42.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.8 kB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numba-0.59.1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.7 MB)
Using cached llvmlite-0.42.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (43.8 MB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      S

/home/lavi/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/lavi/Documents/myprj/VITS-fast-fine-tuning
mkdir: cannot create directory ‘pretrained_models’: File exists
mkdir: cannot create directory ‘video_data’: File exists
mkdir: cannot create directory ‘raw_audio’: File exists
mkdir: cannot create directory ‘denoised_audio’: File exists
mkdir: cannot create directory ‘custom_character_voice’: File exists
mkdir: cannot create directory ‘segmented_character_voice’: File exists


In [2]:
!pip install tensorflow[and-cuda]

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tensorflow[and-cuda] to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 MB 3.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 59.5 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 56.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 48.5 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.7/867.7 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.7/704.7 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 MB 15.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.4 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 MB 12.0 MB/s eta 0:00:0000:0100:0

In [4]:
#@title STEP 1.5 选择预训练模型
#@markdown ###STEP 1.5 选择预训练模型
#@markdown ###Choose pretrained model to start
#@markdown CJE为中日英三语模型，CJ为中日双语模型，C为纯中文模型

#@markdown CJE for Chinese, Japanese & English model，CJ for Chinese & Japanese model
PRETRAINED_MODEL = "CJE" #@param ["CJE","CJ","C"]
# if PRETRAINED_MODEL == "CJ":
#   !wget https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/D_0-p.pth -O ./pretrained_models/D_0.pth
#   !wget https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/G_0-p.pth -O ./pretrained_models/G_0.pth
#   !wget https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/config.json -O ./configs/finetune_speaker.json
# elif PRETRAINED_MODEL == "CJE":
#   !wget https://huggingface.co/spaces/Plachta/VITS-Umamusume-voice-synthesizer/resolve/main/pretrained_models/D_trilingual.pth -O ./pretrained_models/D_0.pth
#   !wget https://huggingface.co/spaces/Plachta/VITS-Umamusume-voice-synthesizer/resolve/main/pretrained_models/G_trilingual.pth -O ./pretrained_models/G_0.pth
#   !wget https://huggingface.co/spaces/Plachta/VITS-Umamusume-voice-synthesizer/resolve/main/configs/uma_trilingual.json -O ./configs/finetune_speaker.json
# elif PRETRAINED_MODEL == "C":
#   !wget https://huggingface.co/datasets/Plachta/sampled_audio4ft/resolve/main/VITS-Chinese/D_0.pth -O ./pretrained_models/D_0.pth
#   !wget https://huggingface.co/datasets/Plachta/sampled_audio4ft/resolve/main/VITS-Chinese/G_0.pth -O ./pretrained_models/G_0.pth
#   !wget https://huggingface.co/datasets/Plachta/sampled_audio4ft/resolve/main/VITS-Chinese/config.json -O ./configs/finetune_speaker.json

In [ ]:
#@title （可选）加载Google云端硬盘 / Mount Google drive
#@title (optional)

#@markdown 加载Google云端硬盘（更快地上传数据集文件）

#@markdown Mount Google drive for faster data upload
from google.colab import drive
drive.mount('/content/drive')

## STEP 2 上传您的角色音频数据
## Upload your character voices
见[数据集上传选项](https://github.com/Plachtaa/VITS-fast-fine-tuning/blob/main/DATA.MD)
See [data upload options](https://github.com/Plachtaa/VITS-fast-fine-tuning/blob/main/DATA_EN.MD)

###STEP 2.1 上传短音频
### Short audio upload

In [ ]:
#@markdown 上传选项1：运行该代码块会出现一个文件上传的入口

#@markdown Upload option 1: Running this code block will prompt you to upload a file.
%run scripts/voice_upload.py --type zip
!unzip ./custom_character_voice/custom_character_voice.zip -d ./custom_character_voice/

In [ ]:
#@markdown 上传选项2：若您装载了Google云端硬盘，可以直接从Google云端硬盘加载文件。将`.zip`文件上传云端硬盘后，在下面填写文件路径：

#@markdown Upload option 2: If you have mounted Google drive, you can load your files from Google drive directly. After uploading your `.zip` file to Google drive, fill in the path to your file below:
ZIP_PATH = "../drive/MyDrive/samples.zip"  #@param {type:"string"}
!cp {ZIP_PATH} ./custom_character_voice/custom_character_voice.zip
!unzip ./custom_character_voice/custom_character_voice.zip -d ./custom_character_voice/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### STEP 2.2 上传长音频 （单个不应长于20分钟）
### Long audio upload

In [ ]:
#@markdown 上传选项1：运行该代码块会出现一个文件上传的入口

#@markdown Upload option 1: Running this code block will prompt you to upload a file.
%run scripts/voice_upload.py --type audio

In [ ]:
#@markdown 上传选项2：若您装载了Google云端硬盘，可以直接从Google云端硬盘加载文件。将所有长音频文件上传至云端硬盘的同一个文件夹下，在下面填写文件夹路径：

#@markdown Upload option 2: If you have mounted Google drive, you can load your files from Google drive directly. Put all the long audios under one folder, and fill in the path to your folder below:
AUDIO_FOLDER_PATH = "/kaggle/input/jp-dataset/utparaphrase512/wav/"  #@param {type:"string"}
!cp {AUDIO_FOLDER_PATH}/* ./raw_audio/

### STEP 2.3 上传视频（单个不应长于20分钟）
### Video upload

In [ ]:
#@markdown 上传选项1：运行该代码块会出现一个文件上传的入口

#@markdown Upload option 1: Running this code block will prompt you to upload a file.
%run scripts/voice_upload.py --type video

In [ ]:
#@markdown 上传选项2：若您装载了Google云端硬盘，可以直接从Google云端硬盘加载文件。将所有视频文件上传至云端硬盘的同一个文件夹下，在下面填写文件夹路径:

#@markdown Upload option 2: If you have mounted Google drive, you can load your files from Google drive directly. Put all the videos under one folder, and fill in the path to your folder below:
VIDEO_FOLDER_PATH = "../drive/MyDrive/videos/"  #@param {type:"string"}
!cp {VIDEO_FOLDER_PATH}/* ./video_data/

In [ ]:
!rm  /content/VITS-fast-fine-tuning/custom_character_voice/*.wav

### STEP 2.4 上传视频链接（单个不应长于20分钟）
### Video link upload

In [ ]:
#@markdown 运行该代码块会出现一个文件上传的入口，上传单个`.txt`文件。若格式正确的话，视频会自动下载并将下载后的文件名打印在下方。

#@markdown Running this code block will prompt you to upload a file.
#@markdown Please upload a single `.txt` file. If you have put the links in the correct format,
#@markdown the videos will be automatically downloaded and displayed below.
%run scripts/download_video.py
!ls ./video_data/

## STEP 3 自动处理所有上传的数据

In [4]:
!mkdir ./custom_character_voice/voice_1
!mkdir ./custom_character_voice/voice_2

In [5]:
!cp /home/lavi/Documents/myprj/jsut_ver1.1/voiceactress100/wav/*.wav ./custom_character_voice/voice_1
!cp /home/lavi/Documents/myprj/Kokoro-Speech-Dataset/output/wavs/*.wav ./custom_character_voice/voice_2

In [9]:
!pip install torch==2.0.0

  Using cached cmake-3.29.2-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 10.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 6.6 MB/s eta 0:00:00m eta 0:00:010:00:02m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 10.3 MB/s eta 0:00:000:00:010:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 11.9 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 11.6 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 12.4 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 19.1 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 26.4 MB/s eta 0:00:00m eta 0:00:0136m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 16.7 MB/s eta 0:00:00m eta 0:00:010:00:0

In [7]:
#@markdown 运行该单元格会对所有上传的数据进行自动去背景音&标注。
#@markdown 由于需要调用Whisper和Demucs，运行时间可能较长。

#@markdown Running this codeblock will perform automatic vocal seperation & annotation.
#@markdown Since this step uses Whisper & Demucs, it may take a while to complete.
# 将所有视频（无论是上传的还是下载的，且必须是.mp4格式）抽取音频
# %run scripts/video2audio.py
# 将所有音频（无论是上传的还是从视频抽取的，必须是.wav格式）去噪
!python scripts/denoise_audio.py
# 分割并标注长音频
# !python scripts/long_audio_transcribe.py --languages "{PRETRAINED_MODEL}" --whisper_size large-v2
# 标注短音频
!python scripts/short_audio_transcribe.py --languages "{PRETRAINED_MODEL}" --whisper_size medium
# 底模采样率可能与辅助数据不同，需要重采样
!python scripts/resample.py

Traceback (most recent call last):
  File "/home/lavi/Documents/myprj/VITS-fast-fine-tuning/scripts/denoise_audio.py", line 16, in <module>
    wav, sr = torchaudio.load(f"./separated/htdemucs/{file}/vocals.wav", frame_offset=0, num_frames=-1, normalize=True,
  File "/home/lavi/anaconda3/envs/VITS/lib/python3.10/site-packages/torchaudio/_backend/utils.py", line 205, in load
    return backend.load(uri, frame_offset, num_frames, normalize, channels_first, format, buffer_size)
  File "/home/lavi/anaconda3/envs/VITS/lib/python3.10/site-packages/torchaudio/_backend/ffmpeg.py", line 297, in load
    return load_audio(uri, frame_offset, num_frames, normalize, channels_first, format)
  File "/home/lavi/anaconda3/envs/VITS/lib/python3.10/site-packages/torchaudio/_backend/ffmpeg.py", line 88, in load_audio
    s = torchaudio.io.StreamReader(src, format, None, buffer_size)
  File "/home/lavi/anaconda3/envs/VITS/lib/python3.10/site-packages/torio/io/_streaming_media_decoder.py", line 526, in __in

#！！！训练质量相关：实验发现目前使用CJ模型+勾选ADD_AUXILIARY，对于中/日均能训练出最好的效果，第一次训练建议默认使用该组合！！！

In [10]:
#@markdown ##STEP 3.5
#@markdown 运行该单元格会生成划分好训练/测试集的最终标注，以及配置文件

#@markdown Running this block will generate final annotations for training & validation, as well as config file.

#@markdown 选择是否加入辅助训练数据：/ Choose whether to add auxiliary data:
ADD_AUXILIARY = False #@param {type:"boolean"}
#@markdown 辅助训练数据是从预训练的大数据集抽样得到的，作用在于防止模型在标注不准确的数据上形成错误映射。

#@markdown Auxiliary data is to prevent overfitting when the audio samples are small or with low quality.

#@markdown 以下情况请勾选：

#@markdown 总样本少于100条/样本质量一般或较差/样本来自爬取的视频

#@markdown 以下情况可以不勾选：

#@markdown 总样本量很大/样本质量很高/希望加速训练/只有二次元角色

# assert(not (ADD_AUXILIARY and PRETRAINED_MODEL != "CJE")), "add auxiliary data is available only available for CJE model!"
if ADD_AUXILIARY:
  %run preprocess_v2.py --add_auxiliary_data True --languages "{PRETRAINED_MODEL}"
else:
  %run preprocess_v2.py --languages "{PRETRAINED_MODEL}"

/home/lavi/anaconda3/envs/VITS/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Downloading: "https://github.com/r9y9/open_jtalk/releases/download/v1.11.1/open_jtalk_dic_utf_8-1.11.tar.gz"


dic.tar.gz: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22.6M/22.6M [00:03<00:00, 6.32MB/s]


Extracting tar file /home/lavi/anaconda3/envs/VITS/lib/python3.10/site-packages/pyopenjtalk/dic.tar.gz
finished


## STEP 4 开始训练

In [9]:
!pip install unidecode pyopenjtalk jamo ko_pron pypinyin cn2an indic_transliteration

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4.3 MB/s eta 0:00:00 MB/s eta 0:00:01:010m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyopenjtalk: filename=pyopenjtalk-0.3.3-cp310-cp310-linux_x86_64.whl size=1181229 sha256=4691ba8726a4faa66478ac02bd4ef21196fe2e97a845ce5203f68cc87f19d13d
  Stored in directory: /home/lavi/.cache/pip/wheels/64/a9/5f/19eedcf7741bcd1494dc8782b6842e433314793492cc9167c9
Successfully built pyopenjtalk


  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tensorflow[and-cuda] to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 MB 3.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 56.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 64.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 62.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.7/867.7 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.7/704.7 MB 1.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 MB 8.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 MB 9.8 MB/s eta 0:00:00:00:0100:01

In [24]:
!pip install tensorboard

In [16]:
%reload_ext tensorboard
%tensorboard --logdir "./OUTPUT_MODEL"

In [18]:
#@markdown #STEP 4 (>=20 min)
#@markdown 开始微调模型。
#@markdown 训练时长取决于你录入/上传的音频总数。

#@markdown 根据声线和样本质量的不同，所需的训练epochs数也不同。

#@markdown 你也可以在Tensorboard中预览合成效果，若效果满意可提前停止。

#@markdown Model fine-tuning
#@markdown Total time cost depends on the number of voices you recorded/uploaded.

#@markdown Best epoch number varies depending on different uploaded voices / sample quality.

#@markdown You can also preview synthezied audio in Tensorboard, it's OK to shut down training manually if you find the quality is satisfying.
# import os
# os.environ['TENSORBOARD_BINARY'] = '/usr/local/bin/tensorboard'

# if os.path.exists("/content/drive/MyDrive/"):
#   !python scripts/rearrange_speaker.py
#   !cp ./finetune_speaker.json ../drive/MyDrive/finetune_speaker.json
#   !cp ./moegoe_config.json ../drive/MyDrive/moegoe_config.json

# %reload_ext tensorboard
# %tensorboard --logdir "./OUTPUT_MODEL"
Maximum_epochs = "200" #@param {type:"string"}
#@markdown 继续之前的模型训练/Continue training from previous checkpoint
CONTINUE = True #@param {type:"boolean"}
if CONTINUE:
  !python finetune_speaker_v2.py -m "./OUTPUT_MODEL" --max_epochs "{Maximum_epochs}" --drop_speaker_embed False --cont True
else:
  !python finetune_speaker_v2.py -m "./OUTPUT_MODEL" --max_epochs "{Maximum_epochs}" --drop_speaker_embed True

2024-05-04 11:49:43.664095: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-04 11:49:43.688345: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 11:49:43.688384: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 11:49:43.689262: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-04 11:49:43.694263: I tensorflow/core/platform/cpu_feature_guar

In [42]:
!zip -r outputModel.zip /kaggle/working/VITS-fast-fine-tuning/OUTPUT_MODEL

  adding: kaggle/working/VITS-fast-fine-tuning/OUTPUT_MODEL/ (stored 0%)
  adding: kaggle/working/VITS-fast-fine-tuning/OUTPUT_MODEL/eval/ (stored 0%)
  adding: kaggle/working/VITS-fast-fine-tuning/OUTPUT_MODEL/eval/events.out.tfevents.1714722831.8560478dfd18.3665.1 (deflated 9%)
  adding: kaggle/working/VITS-fast-fine-tuning/OUTPUT_MODEL/eval/events.out.tfevents.1714724659.8560478dfd18.6059.1 (deflated 16%)
  adding: kaggle/working/VITS-fast-fine-tuning/OUTPUT_MODEL/train.log (deflated 71%)
  adding: kaggle/working/VITS-fast-fine-tuning/OUTPUT_MODEL/config.json (deflated 68%)
  adding: kaggle/working/VITS-fast-fine-tuning/OUTPUT_MODEL/G_1000.pth (deflated 7%)
  adding: kaggle/working/VITS-fast-fine-tuning/OUTPUT_MODEL/G_latest.pth (deflated 7%)
  adding: kaggle/working/VITS-fast-fine-tuning/OUTPUT_MODEL/D_1100.pth (deflated 7%)
  adding: kaggle/working/VITS-fast-fine-tuning/OUTPUT_MODEL/G_1100.pth (deflated 7%)
  adding: kaggle/working/VITS-fast-fine-tuning/OUTPUT_MODEL/githash (defla

In [1]:
#@markdown ### 微调完成后，在这里尝试效果。
#@markdown ### 运行后会输出一个public URL, 点击进入网页版UI以使用模型
#@markdown ### Try out TTS & VC quality here after fine-tuning is finished.
# %cd /kaggle/working/VITS-fast-fine-tuning
!cp ./configs/modified_finetune_speaker.json ./finetune_speaker.json
!gradio VC_inference.py --model_dir /home/lavi/Documents/myprj/VITS-fast-fine-tuning/OUTPUT_MODEL/G_latest.pth --share True


Watching: '/home/lavi/anaconda3/envs/VITS/lib/python3.10/site-packages/gradio', '/home/lavi/Documents/myprj/VITS-fast-fine-tuning'

DEBUG:matplotlib:matplotlib data path: /home/lavi/anaconda3/envs/VITS/lib/python3.10/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/home/lavi/.config/matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.gradio.app:443
DEBUG:matplotlib:CACHEDIR=/home/lavi/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/lavi/.cache/matplotlib/fontlist-v330.json
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/lavi/anaconda3/envs/VITS/lib/python3.10/site-packages/certifi/cacert.pem'
DEBUG:PIL.Image:Importing BlpImagePlugin
DEBUG:PIL.Image:Importing BmpImagePlugin
DEBUG:PIL.Image:Importing BufrStubImagePlugin
DEBUG:PIL.Image:Importing CurImagePlugin
D

# STEP 5 下载模型
## 本地部署方法请见[README](https://github.com/Plachtaa/VITS-fast-fine-tuning/blob/main/README_ZH.md)

In [ ]:
#@markdown ### 下载选项1：运行该单元格，浏览器会自动下载模型和配置文件
#@markdown ### Download option 1: Running this codeblock will download model & config files by your browser.
!python scripts/rearrange_speaker.py
%run scripts/download_model.py

In [ ]:
#@markdown ### 下载选项2：运行该单元格会将模型和配置文件保存到Google云端硬盘
#@markdown ### Download option 2: Running this codeblock will save the mode & config files to your Google drive.
!python scripts/rearrange_speaker.py
!cp ./G_latest.pth ../drive/MyDrive/G_latest.pth
!cp ./finetune_speaker.json ../drive/MyDrive/finetune_speaker.json
!cp ./moegoe_config.json ../drive/MyDrive/moegoe_config.json

In [ ]:
#@markdown ### 运行该单元格会清空所有已上传的样本，需要时可使用
#@markdown ### Running this codeblock will delete all voice samples you have uploaded. Use it if you need.
!rm -rf ./custom_character_voice/*
!rm -rf ./video_data/*
!rm -rf ./raw_audio/*
!rm -rf ./denoised_audio/*
!rm -rf ./segmented_character_voice/*
!rm -rf long_character_anno.txt
!rm -rf short_character_anno.txt

In [ ]:
#@markdown ### 运行该单元格会将切片和标注复制到谷歌云端硬盘根目录下名为`voice_data`的文件夹下以用作其它用途
#@markdown ### Running this codeblock will copy all processed voices & annotations to a folder named `voice_data` under the root of Google Drive for other purpose of usage
!mkdir ../drive/MyDrive/voice_data/
!cp -rf ./custom_character_voice/ ../drive/MyDrive/voice_data/custom_character_voice/
!cp -rf ./segmented_character_voice/ ../drive/MyDrive/voice_data/segmented_character_voice/
!cp long_character_anno.txt ../drive/MyDrive/voice_data/long_character_anno.txt
!cp short_character_anno.txt ../drive/MyDrive/voice_data/short_character_anno.txt